In [1]:
import pandas
import configparser
import psycopg2

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['database']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

In [3]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

In [4]:
table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp18
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

In [5]:
cur.execute(table_sql)
conn.commit()

In [6]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst+junior&start=10'

In [7]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
#print(html_data.decode('utf-8'))

In [8]:
!pip install beautifulsoup4

In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
#print (soup)

In [10]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [11]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [12]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

In [13]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp18(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

In [14]:
df = pandas.read_sql_query('select * from indeed_gp18', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Junior Analyst Apprentice,Alchemy Security LLC.,"Denver, CO",None,About Alchemy Security - *Alchemy Security is ...
1,2,Junior BI Analyst - Must already be based in SF,Puls,"San Francisco, CA 94105 (Financial District area)",None,We are looking for a Junior BI Analyst to join...
2,3,Junior Analyst (Life Science Consulting) SF,Adivo Associates,"San Francisco, CA",None,"We have broad and deep expertise in strategy, ..."
3,4,Insider Threat Cyber Intelligence Analyst-Junior,Mission Essential,"St. Louis, MO",None,• Conduct cyber intelligence analysis.*. Cyber...
4,5,Junior All- Source Intelligence Analyst,BAE Systems,None,None,About BAE Systems Intelligence & Security. BAE...
5,6,Junior Intelligence Analyst,IntelliWare Systems,None,None,IntelliWare is an award-winning company seekin...
6,7,Junior Intelligence Analyst - HLOP10,E3/Sentinel,None,None,E3/Sentinel is seeking a Journeyman Intelligen...
7,8,Junior Business Intelligence Analyst,The Community Company,None,None,NOW HIRING A TECHNICAL JUNIOR BUSINESS INTELLI...
8,9,Junior Intelligence Analyst,"Dynamis, Inc",None,None,Experience or completed training in intelligen...
9,10,Junior Imagery Analyst (Training Provided),ECS Federal LLC,None,$15 an hour,The Junior Imagery Analyst (IA) candidate shou...


In [16]:
cur.close()
conn.close()